# export heat flux
- This script is used to export heat flux variables;
- Simulations: CNTL, ROOF_TV, IMPROAD_TV, WALL_TV, ROOF_IMPROAD_TV, ROOF_IMPROAD_WALL_TV;

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import cftime

In [2]:
path = '/work/n02/n02/yuansun/cesm/'
# surface
sfile = path + 'cesm_inputdata/lnd/clm2/surfdata_map/release-clm5.0.18/surfdata_0.9x1.25_hist_16pfts_Irrig_CMIP6_simyr1850_c190214.nc'
sf = xr.open_dataset(sfile)
#case2
path_2 = path + 'archive/case2/lnd/hist/'
#case3
path_3 = path + 'archive/case3/lnd/hist/'
#case4
path_4 = path + 'archive/case4/lnd/hist/'
#case5
path_5 = path + 'archive/case5/lnd/hist/'
#case6
path_6 = path + 'archive/case6/lnd/hist/'

# dynamic roof
sfile_dyn_rf = path + 'cesm_inputdata/lnd/clm2/urbandata/CLM50_DynUrbanAlbedoRoof_YuanSun_2023_0.9x1.25_simyr1849-2106_c20231005.nc'
dyn_rf = xr.open_dataset(sfile_dyn_rf).sel(time = slice('2015-01-01 00:00:00', '2100-01-01 00:00:00'))

start_year = 2015
end_year = 2100

dyn_rf['time'] = dyn_rf['time'].dt.year

In [3]:
lsmmask = np.any(sf['PCT_URBAN'] != 0, axis=0)
mask = lsmmask.rename({'lsmlat': 'lat', 'lsmlon': 'lon'})

roof = (sf['WTLUNIT_ROOF']).where(lsmmask)
perroad = ((1 - sf['WTLUNIT_ROOF']) * sf['WTROAD_PERV']).where(lsmmask)
improad = ((1 - sf['WTLUNIT_ROOF']) * (1-sf['WTROAD_PERV'])).where(lsmmask)

alb_perroad = (sf['ALB_PERROAD_DIF'][0,:,:,:]).where(lsmmask)
alb_improad = (sf['ALB_IMPROAD_DIF'][0,:,:,:]).where(lsmmask)
alb_rf_tbd = (dyn_rf['dyn_alb_roof_TBD']).where(mask)
alb_rf_hd = (dyn_rf['dyn_alb_roof_HD']).where(mask)
alb_rf_md = (dyn_rf['dyn_alb_roof_MD']).where(mask)

In [4]:
arearoof = roof.mean()
areaperroad = perroad.mean()
areaimproad = improad.mean() 
print(arearoof, areaperroad, areaimproad)

<xarray.DataArray 'WTLUNIT_ROOF' ()>
array(0.54381721) <xarray.DataArray ()>
array(0.23361668) <xarray.DataArray ()>
array(0.22256611)


In [5]:
rf_tbd = (roof[0,:,:].rename({'lsmlat': 'lat', 'lsmlon': 'lon'})) * alb_rf_tbd 
rf_hd  = (roof[1,:,:].rename({'lsmlat': 'lat', 'lsmlon': 'lon'})) * alb_rf_hd 
rf_md  = (roof[2,:,:].rename({'lsmlat': 'lat', 'lsmlon': 'lon'})) * alb_rf_md
rf = xr.concat([rf_tbd, rf_hd, rf_md], dim='numurbl')
#rf # numurbl: 3lat: 192lon: 288time: 86

In [6]:
ipd = (perroad * alb_perroad).rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
id = (improad * alb_improad).rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
#alb = (rf + ipd + id)/(roof +perroad + improad).rename({'lsmlat': 'lat', 'lsmlon': 'lon'})
alb = (rf/roof.rename({'lsmlat': 'lat', 'lsmlon': 'lon'}))
alb # numurbl: 3lat: 192lon: 288time: 86

<xarray.DataArray (numurbl: 3, lat: 192, lon: 288, time: 86)>
array([[[[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
...
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]],

        [[nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         ...,
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan],
         [nan, nan, nan, ..., nan, nan, nan]]]])
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.7
  * time     (time) int64 2015 2016 2017 2018 2019 ... 2096 2097 2098 2099 2100
Dimensions without coordinates: numurbl

In [7]:
df_alb = alb.mean(axis = 0).to_dataframe(name='alb').reset_index()
df_alb

,lat,lon,time,alb
0,-90.0,0.00,2015,NaN
1,-90.0,0.00,2016,NaN
2,-90.0,0.00,2017,NaN
3,-90.0,0.00,2018,NaN
4,-90.0,0.00,2019,NaN
...,...,...,...,...
4755451,90.0,358.75,2096,NaN
4755452,90.0,358.75,2097,NaN
4755453,90.0,358.75,2098,NaN
4755454,90.0,358.75,2099,NaN


In [157]:
df_alb.max()

lat       90.00
lon      358.75
time    2100.00
alb        0.90
dtype: float64

In [8]:
df_alb.rename(columns={'time': 'year'}, inplace=True)
df_alb['lsmlon'] = (df_alb['lon']/1.25).astype(int)
unique_lat_values = df_alb['lat'].unique()
lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
df_alb['lsmlat'] = df_alb['lat'].map(lat_mapping)
df_alb

,lat,lon,year,alb,lsmlon,lsmlat
0,-90.0,0.00,2015,NaN,0,0
1,-90.0,0.00,2016,NaN,0,0
2,-90.0,0.00,2017,NaN,0,0
3,-90.0,0.00,2018,NaN,0,0
4,-90.0,0.00,2019,NaN,0,0
...,...,...,...,...,...,...
4755451,90.0,358.75,2096,NaN,286,191
4755452,90.0,358.75,2097,NaN,286,191
4755453,90.0,358.75,2098,NaN,286,191
4755454,90.0,358.75,2099,NaN,286,191


In [9]:
sort = df_alb.dropna()
sort

,lat,lon,year,alb,lsmlon,lsmlat
936454,-55.130890,291.25,2015,0.394333,233,37
936455,-55.130890,291.25,2016,0.404333,233,37
936456,-55.130890,291.25,2017,0.414333,233,37
936457,-55.130890,291.25,2018,0.424333,233,37
936458,-55.130890,291.25,2019,0.434333,233,37
...,...,...,...,...,...,...
4212791,70.209424,31.25,2096,0.900000,25,170
4212792,70.209424,31.25,2097,0.900000,25,170
4212793,70.209424,31.25,2098,0.900000,25,170
4212794,70.209424,31.25,2099,0.900000,25,170


In [11]:
ALB = sort.groupby('year').mean().reset_index()['alb']
ALB.to_csv('../UHI/gridcell_alb_dynroof.csv')

In [10]:
# check delta_albedo
df_sorted = sort.sort_values(by=['lat', 'lon', 'year'])
df_sorted['delta'] = df_sorted.groupby(['lat', 'lon'])['alb'].diff()
df_sorted

,lat,lon,year,alb,lsmlon,lsmlat,delta
936454,-55.130890,291.25,2015,0.309167,233,37,NaN
936455,-55.130890,291.25,2016,0.315833,233,37,0.006667
936456,-55.130890,291.25,2017,0.322500,233,37,0.006667
936457,-55.130890,291.25,2018,0.329167,233,37,0.006667
936458,-55.130890,291.25,2019,0.335833,233,37,0.006667
...,...,...,...,...,...,...,...
4212791,70.209424,31.25,2096,0.562167,25,170,0.000000
4212792,70.209424,31.25,2097,0.562167,25,170,0.000000
4212793,70.209424,31.25,2098,0.562167,25,170,0.000000
4212794,70.209424,31.25,2099,0.562167,25,170,0.000000


In [11]:
case2_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case2_flux = pd.concat([case2_flux, df], ignore_index=True)

In [365]:
case2_flux

,lat,lon,FIRA_U,year,lsmlon,lsmlat,FSA_U,FSH_U,EFLX_LH_TOT_U,FGR_U
0,-55.130890,291.25,21.685835,2015,233,37,62.550735,2.463449,39.729191,-0.133372
1,-54.188480,292.50,31.818651,2015,234,38,80.481018,18.697359,32.053322,-1.362421
2,-53.246075,288.75,23.530861,2015,231,39,64.388817,3.854504,38.917187,-0.712562
3,-53.246075,291.25,33.501308,2015,233,39,81.254112,20.069777,29.788179,-1.453790
4,-51.361256,291.25,45.561325,2015,233,41,101.318092,32.175941,23.949677,-0.139984
...,...,...,...,...,...,...,...,...,...,...
360480,69.267014,32.50,31.259878,2099,26,169,44.533813,16.100861,15.078340,-13.352802
360481,69.267014,33.75,29.729136,2099,27,169,43.593094,11.749278,17.140741,-11.311886
360482,69.267014,87.50,29.782660,2099,70,169,53.420910,31.040018,13.927350,-16.305197
360483,69.267014,88.75,30.175827,2099,71,169,51.676563,40.280163,10.795192,-21.738813


In [17]:
merged_df = pd.merge(case2_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case2_export_annual.csv')

In [20]:
# jja
case2_flux_jja = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        df = ds['FIRA_U'][6:9,:,:].mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = ds['FSA_U'][6:9,:,:].mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSDS'] = (ds['FSDS'][6:9,:,:]).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case2_flux_jja = pd.concat([case2_flux_jja, df], ignore_index=True)

In [21]:
merged_df_jja = pd.merge(case2_flux_jja, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df_jja.to_csv('case2_export_jja.csv')

In [19]:
# djf
case2_flux_djf = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    fn_1 = 'case2.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_2+fn)):
        ds = xr.open_dataset(path_2+fn)
        ds_1 = xr.open_dataset(path_2+fn)
        df = xr.concat([ds['FIRA_U'].isel(time=11),ds_1['FIRA_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = xr.concat([ds['FSA_U'].isel(time=11),ds_1['FSA_U'].isel(time=slice(0, 2))], dim='time').mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSDS'] = xr.concat([ds['FSDS'].isel(time=11),ds_1['FSDS'].isel(time=slice(0, 2))], dim='time').where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case2_flux_djf = pd.concat([case2_flux_djf, df], ignore_index=True)

In [22]:
merged_df_djf = pd.merge(case2_flux_djf, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df_djf.to_csv('case2_export_djf.csv')

In [23]:
# case3
case3_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case3.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_3+fn)):
        ds = xr.open_dataset(path_3+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case3_flux = pd.concat([case3_flux, df], ignore_index=True)

In [24]:
merged_df = pd.merge(case3_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case3_export_annual.csv')

In [25]:
# case4
case4_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case4.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_4+fn)):
        ds = xr.open_dataset(path_4+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case4_flux = pd.concat([case4_flux, df], ignore_index=True)

In [26]:
merged_df = pd.merge(case4_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case4_export_annual.csv')

In [27]:
# case5
case5_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case5.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_5+fn)):
        ds = xr.open_dataset(path_5+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case5_flux = pd.concat([case5_flux, df], ignore_index=True)

In [28]:
merged_df = pd.merge(case5_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case5_export_annual.csv')

In [31]:
# case6
case6_flux = pd.DataFrame()
for year in range(start_year,end_year):
    fn = 'case6.clm2.h1.' + '%04.0f' % year + '-02-01-00000.nc'
    if(os.path.exists(path_6+fn)):
        ds = xr.open_dataset(path_6+fn)
        df = (ds['FIRA_U']).mean('time').to_dataframe(name='FIRA_U').reset_index()
        df['year'] = year
        df['lsmlon'] = (df['lon']/1.25).astype(int)
        unique_lat_values = df['lat'].unique()
        lat_mapping = {lat: index for index, lat in enumerate(unique_lat_values)}
        df['lsmlat'] = df['lat'].map(lat_mapping)
        df = df.dropna()
        df['FSA_U'] = (ds['FSA_U']).mean('time').to_dataframe(name='FSA_U').reset_index()['FSA_U']
        df['FSDS'] = (ds['FSDS']).where(mask).mean('time').to_dataframe(name='FSDS').reset_index()['FSDS']
        case6_flux = pd.concat([case6_flux, df], ignore_index=True)

In [32]:
merged_df = pd.merge(case6_flux, sort, on=['lsmlat', 'lsmlon', 'year'], how='inner').dropna()
merged_df.to_csv('case6_export_annual.csv')